## accesso al daily firstrate

In [ ]:
from revelation.data.loading import (
    Catalog,
    firstrate_dirname,
    firstrate_filename,
    CSVPreset,
)
import pandas as pd
from revelation.visualization.visualization import hvplot_ohlc

catalog = Catalog()
symbol_name = firstrate_filename(resolution="min", year=2023)
ohlc: pd.DataFrame = catalog.get_csv(
    catalog.raw_directory
    / "csv/firstrate"
    / firstrate_dirname(resolution="m")
    / symbol_name,
    preset=CSVPreset.FIRSTRATE,
)

In [52]:
ohlc

,open,high,low,close,volume,open_interest
timestamp,,,,,,
2021-07-13 16:59:00-04:00,1.19500,1.19500,1.19500,1.19500,1,NaN
2021-09-23 05:23:00-04:00,1.18900,1.18900,1.18900,1.18900,1,NaN
2021-11-03 02:14:00-04:00,1.17430,1.17430,1.17430,1.17430,1,NaN
2021-11-03 02:15:00-04:00,1.17415,1.17415,1.17415,1.17415,1,NaN
2021-11-03 03:08:00-04:00,1.17415,1.17415,1.17415,1.17415,2,NaN
...,...,...,...,...,...,...
2023-03-13 10:00:00-04:00,1.06980,1.06980,1.06980,1.06980,4,NaN
2023-03-13 10:05:00-04:00,1.07115,1.07130,1.07115,1.07130,20,NaN
2023-03-13 10:06:00-04:00,1.07145,1.07145,1.07145,1.07145,23,NaN


In [56]:
from revelation.data.enums import DayOfWeek
from revelation.data.aggregation import subsample_ohlc


ohlc_sub = subsample_ohlc(ohlc, timeframe="1h")
# solo dal 2024
ohlc_sub = ohlc_sub.dropna(subset=["open", "high", "low", "close"], how="all")["2023":]

for day in DayOfWeek:

    mondays: pd.DataFrame = ohlc_sub[ohlc_sub.index.dayofweek == day]
    daily_range = (
        mondays["high"].resample("D").max() - mondays["low"].resample("D").min()
    )
    print(day.name, round(daily_range.dropna().mean(), 4))

MON 0.0073
TUE 0.0095
WED 0.0082
THU 0.0089
FRI 0.0096
SAT nan
SUN 0.0033


In [57]:
hvplot_ohlc(ohlc_sub, symbol_name)

:Overlay
   .Segments.I   :Segments   [timestamp,low,timestamp,high]   (open,high,low,close,lbound,ubound)
   .Rectangles.I :Rectangles   [lbound,open,ubound,close]   (open,high,low,close,lbound,ubound)

## accesso ai dati tradingview

In [5]:
import vectorbtpro as vbt
from revelation.time_utils import STANDARD_TIMEZONE
from dotenv import load_dotenv

load_dotenv()
import os

# tradingview_client = vbt.TVData.resolve_client(
#     # username=os.getenv("TRADINGVIEW_USERNAME"),
#     # password=os.getenv("TRADINGVIEW_PASSWORD"),
#     auth_token=os.getenv("TRADINGVIEW_AUTH_TOKEN"),
# )

In [6]:
data = vbt.TVData.fetch_symbol(
    "CME:6EH2024",
    timeframe="1 hour",  # client=tradingview_client
)
data[0]

,Open,High,Low,Close,Volume
datetime,,,,,
2021-02-05 20:00:00+00:00,1.24310,1.24310,1.24310,1.24310,1.0
2021-02-08 20:00:00+00:00,1.24470,1.24470,1.24470,1.24470,1.0
2021-02-09 20:00:00+00:00,1.25070,1.25070,1.25070,1.25070,1.0
2021-02-10 20:00:00+00:00,1.25110,1.25110,1.25110,1.25110,1.0
2021-02-11 20:00:00+00:00,1.25160,1.25160,1.25160,1.25160,1.0
...,...,...,...,...,...
2024-03-18 10:00:00+00:00,1.08995,1.09010,1.08970,1.09000,52.0
2024-03-18 11:00:00+00:00,1.09015,1.09045,1.09015,1.09035,103.0
2024-03-18 12:00:00+00:00,1.09025,1.09045,1.08915,1.08915,101.0


In [7]:
df: pd.DataFrame = data[0].copy()
df.index = df.index.tz_convert(STANDARD_TIMEZONE)
df.columns = df.columns.str.lower()
df

,open,high,low,close,volume
datetime,,,,,
2021-02-05 15:00:00-05:00,1.24310,1.24310,1.24310,1.24310,1.0
2021-02-08 15:00:00-05:00,1.24470,1.24470,1.24470,1.24470,1.0
2021-02-09 15:00:00-05:00,1.25070,1.25070,1.25070,1.25070,1.0
2021-02-10 15:00:00-05:00,1.25110,1.25110,1.25110,1.25110,1.0
2021-02-11 15:00:00-05:00,1.25160,1.25160,1.25160,1.25160,1.0
...,...,...,...,...,...
2024-03-18 06:00:00-04:00,1.08995,1.09010,1.08970,1.09000,52.0
2024-03-18 07:00:00-04:00,1.09015,1.09045,1.09015,1.09035,103.0
2024-03-18 08:00:00-04:00,1.09025,1.09045,1.08915,1.08915,101.0


In [8]:
import json
import uuid
from typing import Any

import pandas as pd
from IPython.display import HTML, display, Markdown

__all__ = ["plot"]


# TODO un to_json probabilmente è più efficiente
def _to_lw_format(df: pd.DataFrame) -> list[dict[str, Any]]:
    """Convert OHLC DataFrame to list-of-dicts format expected by lightweight-charts."""
    out = [
        {
            "time": int(pd.Timestamp(ts).timestamp()),  # seconds since Epoch
            "open": float(row["open"]),
            "high": float(row["high"]),
            "low": float(row["low"]),
            "close": float(row["close"]),
        }
        for ts, row in df.iterrows()
    ]
    return out


def plot(
    dataframe: pd.DataFrame,
    *,
    height: int = 400,
    width: str | int = "100%",
) -> None:
    # --- validation ---------------------------------------------------------
    required = {"open", "high", "low", "close"}
    if not required.issubset(dataframe.columns):
        raise ValueError(f"DataFrame must contain columns: {required}")
    if not isinstance(dataframe.index, pd.DatetimeIndex):
        raise TypeError("DataFrame index must be a DateTimeIndex")

    # --- prepare data & ids --------------------------------------------------
    candles_json = json.dumps(_to_lw_format(dataframe), separators=(",", ":"))
    div_id = f"lwch_{uuid.uuid4().hex}"

    # --- HTML/JS payload -----------------------------------------------------
    html = f"""
    <div id="{div_id}" style="width:{width};height:{height}px;"></div>
    <script type="text/javascript">
    // Helper to load a script only once
    function loadScript(src) {{
        return new Promise((resolve, reject) => {{
            if (document.querySelector(`script[src="${{src}}"]`)) {{ resolve(); return; }}
            const s = document.createElement('script');
            s.src = src;
            s.onload = resolve;
            s.onerror = reject;
            document.head.appendChild(s);
        }});
    }}

    (async () => {{
        // Load dependencies (Vue 3 + Lightweight-Charts)
        await loadScript('https://unpkg.com/vue@3/dist/vue.global.prod.js');
        await loadScript('https://unpkg.com/lightweight-charts@4.1.0/dist/lightweight-charts.standalone.production.js');

        const {{ createApp, onMounted, ref }} = Vue;

        const App = {{
            setup() {{
                const container = ref(null);

                onMounted(() => {{
                    const chart = LightweightCharts.createChart(container.value, {{
                        width: container.value.clientWidth,
                        height: {height},
                        layout: {{
                            background: {{ type: 'solid', color: '#ffffff' }},
                            textColor: '#333',
                        }},
                        grid: {{
                            vertLines: {{ color: '#f0f3fa' }},
                            horzLines: {{ color: '#f0f3fa' }},
                        }},
                    }});
                    const series = chart.addCandlestickSeries();
                    series.setData({candles_json});

                    // Keep chart responsive
                    new ResizeObserver(entries => {{
                        for (const entry of entries) {{
                            chart.applyOptions({{ width: entry.contentRect.width }});
                        }}
                    }}).observe(container.value);
                }});

                return {{ container }};
            }},
            template: '<div ref="container" style="width:100%;height:100%;"></div>',
        }};

        createApp(App).mount('#{div_id}');
    }})();
    </script>
    """

    display(HTML(html))

voglio prendere un df, inserirlo in una funzione plot e questa deve comunicare con vue tv
cosi da aprire un grafico immediatamente, non mi interessa se direttamente con jupyter output o webpage.

il df deve essere in memory

prova disegno box, orari di tv chart sono comunque errati

In [9]:
# BUG gli orari sono comunque sbagliati
chart = set_chart(df)
chart.box(
    df.index[-3],
    df["low"].iloc[-3],
    df.index[-1],
    df["high"].iloc[-1],
)
chart.load()

NameError: name 'set_chart' is not defined